In [1]:
from pytezos import pytezos
import requests
import json
from pandas import json_normalize
import time
from random import choice
from pprint import pprint
from random import randint
from pytezos.rpc.errors import RpcError, MichelsonError

address = 'KT1JPehwEkAhg1z5RVA8wAccYbDgF2E2itDv'
# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
# pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')
pytezos = pytezos.using(shell='https://granadanet.smartpy.io/')

In [2]:
from os.path import join
KEYS_DIR = '../../../juster-maker/keys'

In [3]:
keys = dict(
    a = pytezos.using(key=join(KEYS_DIR, 'tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu.json')).contract(address),
    b = pytezos.using(key=join(KEYS_DIR, 'tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK.json')).contract(address),
    c = pytezos.using(key=join(KEYS_DIR, 'tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y.json')).contract(address),
    d = pytezos.using(key=join(KEYS_DIR, 'tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6.json')).contract(address),
    e = pytezos.using(key=join(KEYS_DIR, 'tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM.json')).contract(address)
)

In [4]:
last_used = {}

def pick_random_free_key(keys, last_used, time_to_use_again=60):
    while True:
        random_key = choice(list(keys.keys()))
        current_time = time.time()
        elapsed_time = current_time - last_used.get(random_key, 0)
        if elapsed_time > time_to_use_again:
            last_used[random_key] = current_time
            return random_key
        else:
            time.sleep(1)

def wait_while_key_can_be_used(key, last_used):
    the_only_key = {key: keys[key]}
    return pick_random_free_key(the_only_key, last_used)

In [5]:
CURRENCY_PAIRS = [
    'BAT-USDC', 'BTC-USD', 'COMP-USD', 'DAI-USDC', 'ETH-USD',
    'KNC-USD', 'LINK-USD', 'REP-USD', 'ZRX-USD',
    # 'XTZ-USD', 
]

# CURRENCY_PAIRS = ['XTZ-USD']

TARGET_DYNAMICS = [
    1_000_000, 1_000_000, 1_000_000, 1_000_000, 1_000_000,
    950_000, 950_000, 950_000,
    1_050_000, 1_050_000, 1_050_000,
    900_000,
    1_000_000
]

RUN_TIME = int(time.time())

# picking nearest available hour:
RUN_TIME = RUN_TIME - RUN_TIME % 300
BETS_CLOSE_TIME = [RUN_TIME + 3600]
# BETS_CLOSE_TIME = [RUN_TIME + 604800*2]  # two weeks to collect bets
MEASURE_PERIOD = [300, 600, 1200]
LIQUIDITY_PERCENT = [10_000, 25_000, 50_000, 100_000]
MEASURE_START_FEE = [100_000, 200_000, 500_000]
EXPIRATION_FEE = [100_000, 200_000, 500_000]

LIQUIDITY_VALUE = [1_000_000, 2_000_000, 5_000_000, 10_000_000]
BET_VALUE = [100_000, 250_000, 500_000, 1_000_000]

EVENTS_COUNT = 5

# TODO finish test with mock oracle complete and run
    - otherwise need to write why I was not succeeded to run them?
    /chains/main/blocks/BMWKwbv4HFjXJoKmzEmS1bYWoismCNmzvp5KVoxTqbeHioYzdwr/context/constants

# TODO why does tests with forceMajeure fails in simulation mode?
    *** pytezos.rpc.node.RpcError: ('Not found: '
     '/chains/main/blocks/BMJHK5gc3XPMUVqscu2aiCAjtJJZe8BHqP9XszPJdtWc9EWxJzT/context/constants',)


In [38]:
def repeat_to_succeed_wrapper(func):
    max_attempts = 10

    def wrapped(*args, **kwargs):
        for attempt in range(max_attempts):
            try:
                return func(*args, **kwargs)
            except MichelsonError as e:
                print(f'catched MichelsonError ({attempt}) {e}')
                time.sleep(1)
            except RpcError as e:
                print(f'catched RpcError ({attempt}) {e}')
                time.sleep(1)

    return wrapped


@repeat_to_succeed_wrapper
def create_random_event():
    event_params = {
        'currencyPair': choice(CURRENCY_PAIRS),
        'targetDynamics': choice(TARGET_DYNAMICS),
        'betsCloseTime': choice(BETS_CLOSE_TIME),
        'measurePeriod': choice(MEASURE_PERIOD),
        'liquidityPercent': choice(LIQUIDITY_PERCENT),
    }

    key = pick_random_free_key(keys, last_used)
    print(f'creating event with {key}:')
    pprint(event_params)

    fees = (
        keys[key].storage()['config']['expirationFee']
        + keys[key].storage()['config']['measureStartFee'])

    keys[key].newEvent(event_params).with_amount(fees).as_transaction().autofill().sign().inject(_async=False)
    return event_params


@repeat_to_succeed_wrapper
def provide_liquidity(event_id):
    liquidity_value = choice(LIQUIDITY_VALUE)
    pl_params = {
        'eventId': event_id,
        'expectedRatioAboveEq': choice([1, 2, 3, 4]),
        'expectedRatioBelow': choice([1, 2, 3, 4]),
        'maxSlippage': 10_000_000_000,
    }

    key = pick_random_free_key(keys, last_used)
    print(f'providing liquidity for {event_id} with {key}:')

    transaction = keys[key].provideLiquidity(pl_params).with_amount(liquidity_value).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


@repeat_to_succeed_wrapper
def bet(event_id):
    bet_value = choice(BET_VALUE)
    bet_choice = choice(['aboveEq', 'below'])
    bet_params = {
        'bet': bet_choice,
        'eventId': event_id,
        'minimalWinAmount': bet_value
    }

    key = pick_random_free_key(keys, last_used)
    print(f'bet "{bet_choice}" for {event_id} with {key}:')

    transaction = keys[key].bet(bet_params).with_amount(bet_value).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


@repeat_to_succeed_wrapper
def start_measurement(event_id):
    key = pick_random_free_key(keys, last_used)
    print(f'starting measurement for {event_id} with {key}:')

    transaction = keys[key].startMeasurement(event_id).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


@repeat_to_succeed_wrapper
def close(event_id):
    key = pick_random_free_key(keys, last_used)
    print(f'closing for {event_id} with {key}:')

    transaction = keys[key].close(event_id).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


def withdraw(event_id, key):
    print(f'withdrawing for {event_id} with {key}:')

    params = {'eventId': event_id, 'participantAddress': keys[key].key.public_key_hash()}
    transaction = keys[key].withdraw(params).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


@repeat_to_succeed_wrapper
def trigger_force_majeure(event_id):
    key = pick_random_free_key(keys, last_used)
    transaction = keys[key].triggerForceMajeure(event_id).as_transaction()
    transaction.autofill().sign().inject(_async=False)
    print(f'event: {event_id} triggered by key: {key}')


def wait_until(time_until_wait):
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'
    return 'ready'

In [39]:
# event_params = keys['a'].storage['events'][6]()

In [40]:
'''
def get_event_if_created(event_id):
    try:
        event_params = keys['a'].storage['events'][event_id]()
    except KeyError:
        return
'''

"\ndef get_event_if_created(event_id):\n    try:\n        event_params = keys['a'].storage['events'][event_id]()\n    except KeyError:\n        return\n"

In [61]:
ORACLE_DELAY = 7*60  # starting calling oracle after this time
BET_CLOSE_LAG = 4*60
WAIT_TRANSACTION_LAG = 90

# TODO: split event to actions, yield after each action, if action is not succeed - yield and then repeat action
def event_pipeline(event_id):

    storage = keys['a'].storage()

    # checking if event already created:
    # (the case with event in the future is legit because events created before blockchain is updated)
    if event_id >= storage['nextEventId']:
        event_params = create_random_event()
        yield event_params
        yield from wait_until(time.time() + WAIT_TRANSACTION_LAG)

    try:
        event_params = keys['a'].storage['events'][event_id]()
    except KeyError:
        print(f'catched KeyError in event_pipeline {event_id}, looks like this event already end')
        return

    # providing first liquidity if needed:
    if event_params['poolAboveEq'] == 0:
        yield provide_liquidity(event_id)
        yield from wait_until(time.time() + WAIT_TRANSACTION_LAG)

    # making random bets and adding random liquidity:
    end_time = event_params['betsCloseTime'] - BET_CLOSE_LAG

    while time.time() < end_time:
        next_action = choice([provide_liquidity, bet, bet, bet])
        yield next_action(event_id)

    # waiting for bets close time:
    yield from wait_until(event_params['betsCloseTime'] + ORACLE_DELAY)

    if event_params['measureOracleStartTime'] is None:
        yield start_measurement(event_id)
        yield from wait_until(time.time() + WAIT_TRANSACTION_LAG)

    # waiting for close time:
    updated_event_params = keys['a'].storage['events'][event_id]()
    time_until_wait = (
        updated_event_params['measureOracleStartTime']
        + ORACLE_DELAY
        + event_params['measurePeriod']
    )
    yield from wait_until(time_until_wait)

    if event_params['closedOracleTime'] is None:
        yield close(event_id)
        yield from wait_until(time.time() + WAIT_TRANSACTION_LAG)

    for key in keys:
        # blocking waiting (anyway no one can use keys if they in transactions):
        try:
            wait_while_key_can_be_used(key, last_used)
            yield withdraw(event_id, key)
        except Exception as e:
            print('error', e)

    yield 'finished'

In [62]:
next_event_id = keys['a'].storage['nextEventId']()
# next_event_id = 22
next_event_id

5

In [63]:
pipelines = [event_pipeline(k) for k in [0, 1, 2, 3, 4]]

In [64]:
# keys['a'].storage['events'][4]()

In [65]:
while pipelines:
    for n in range(len(pipelines)):
        try:
            next(pipelines[n])
        except StopIteration:
            # should work, but will skips one pipeline when popping:
            pipelines.pop(n)
        except IndexError:
            continue

catched KeyError in event_pipeline 0, looks like this event already end
catched KeyError in event_pipeline 3, looks like this event already end
catched KeyError in event_pipeline 1, looks like this event already end
withdrawing for 4 with a:
error ({'id': 'proto.010-PtGRANAD.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 7853,
  'with': {'string': 'Participant not found'}},)
withdrawing for 4 with b:
error ({'id': 'proto.010-PtGRANAD.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 7853,
  'with': {'string': 'Participant not found'}},)
withdrawing for 4 with c:
error ({'id': 'proto.010-PtGRANAD.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 7853,
  'with': {'string': 'Participant not found'}},)
withdrawing for 4 with d:
withdrawing for 4 with e:
closing for 2 with a:
withdrawing for 2 with a:
withdrawing for 2 with b:
withdrawing for 2 with c:
withdrawing for 2 with d:
withdrawing for 2 with e:


In [66]:
# trigger_force_majeure(5)
_ = """
for event_id in [3, 4, 5, 6, 7, 8]:
    for key in keys:
        # blocking waiting (anyway no one can use keys if they in transactions):
        try:
            wait_while_key_can_be_used(key, last_used)
            withdraw(event_id, key)
        except Exception as e:
            print('error', e)
"""

In [28]:
response = requests.get(f'https://api.edo2net.tzkt.io/v1/contracts/{address}/storage/history?limit=100')
data = json.loads(response.text)
df = json_normalize(data)

In [67]:
opg = keys['a'].claimRetainedProfits().send()